Import library and preprocess data

In [50]:
#Import library and read data
import pandas as pd
import numpy as np
col_list = ['User', "Longitude", "Latitude", "Date", "Time", "DateNumber"]
df = pd.read_csv("/content/drive/MyDrive/dataset/all_data.csv", usecols=col_list)
df

<ipython-input-50-87bc3c17b500>:5: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/dataset/all_data.csv", usecols=col_list)


,User,Latitude,Longitude,DateNumber,Date,Time
0,0,39.984702,116.318417,39744.120185,2008-10-23,02:53:04
1,0,39.984683,116.318450,39744.120255,2008-10-23,02:53:10
2,0,39.984686,116.318417,39744.120313,2008-10-23,02:53:15
3,0,39.984688,116.318385,39744.120370,2008-10-23,02:53:20
4,0,39.984655,116.318263,39744.120428,2008-10-23,02:53:25
...,...,...,...,...,...,...
24708581,181,40.914267,111.710333,39521.153670,3/14/2008,3:41:17
24708582,181,40.912467,111.710667,39521.154880,3/14/2008,3:43:02
24708583,181,40.911517,111.711317,39521.155190,3/14/2008,3:43:28
24708584,181,40.910933,111.711617,39521.155320,3/14/2008,3:43:40


In [51]:
#Drop Na and convert 'User' column to the same data type (float)
df = df.dropna()
df['User'] = df['User'].astype(float)
df = df.round(3)

<ipython-input-51-1b06202e9fb5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['User'] = df['User'].astype(float)


In [52]:
from geopy.geocoders import Nominatim
from geopy.point import Point
def get_location_type(lat, lon):
    geolocator = Nominatim(user_agent='test')
    try:
      location = geolocator.reverse(Point(lat, lon), timeout=None)
      return location.raw['type']
    except:
      return None


Trên toàn bộ tệp dữ liệu - xác định các địa điểm được ghi nhận nhiều nhất


In [ ]:
#Location Frequency
MostFrequentLocation = (df.groupby(['Latitude', 'Longitude']).size().sort_values(ascending=False).reset_index(name='count'))
MostFrequentLocation

,Latitude,Longitude,count
0,39.973,116.334,94096
1,40.008,116.319,88279
2,39.985,116.353,81809
3,39.975,116.331,72316
4,40.000,116.327,70939
...,...,...,...
673560,36.396,119.796,1
673561,36.396,119.798,1
673562,36.396,119.801,1
673563,36.396,119.802,1


In [ ]:
MostFrequentLocation = MostFrequentLocation[MostFrequentLocation['count'] > 10000]
MostFrequentLocation

,Latitude,Longitude,count
0,39.973,116.334,94096
1,40.008,116.319,88279
2,39.985,116.353,81809
3,39.975,116.331,72316
4,40.000,116.327,70939
...,...,...,...
224,39.991,116.329,10112
225,39.975,116.303,10093
226,39.995,116.326,10049
227,39.977,116.310,10028


In [ ]:
#Get type of location from "nominatim" API
for i in range(len(MostFrequentLocation)):
  MostFrequentLocation.loc[i, 'Location_type'] = get_location_type(MostFrequentLocation.loc[i, 'Latitude'], MostFrequentLocation.loc[i, 'Longitude'])
MostFrequentLocation

<ipython-input-6-8a46ecdc2ffc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MostFrequentLocation.loc[i, 'Location_type'] = get_location_type(MostFrequentLocation.loc[i, 'Latitude'], MostFrequentLocation.loc[i, 'Longitude'])


,Latitude,Longitude,count,Location_type
0,39.973,116.334,94096,residential
1,40.008,116.319,88279,residential
2,39.985,116.353,81809,university
3,39.975,116.331,72316,yes
4,40.000,116.327,70939,unclassified
...,...,...,...,...
224,39.991,116.329,10112,cafe
225,39.975,116.303,10093,tertiary
226,39.995,116.326,10049,telecommunication
227,39.977,116.310,10028,mall


In [ ]:
MostFrequentLocation['Location_type'].unique()

array(['residential', 'university', 'yes', 'unclassified', 'tertiary',
       'primary', 'house', 'it', 'kindergarten', 'parking', 'secondary',
       'massage', 'research_institute', 'school', 'hospital', 'police',
       'bureau_de_change', 'bicycle_parking', 'bakery', 'parcel_locker',
       'atm', 'government', 'cycleway', 'parking_entrance', 'bridge',
       'post_box', 'apartments', 'mall', 'hotel', 'research', 'pitch',
       'surveillance', 'bus_stop', 'cinema', 'cafe', 'artwork',
       'fast_food', 'vending_machine', 'stop', 'wastewater_plant',
       'commercial', 'restaurant', 'telecommunication'], dtype=object)

In [ ]:
MostFrequentLocation['Location_type'].nunique()

43

In [ ]:
MostFrequentLocation.to_csv('/content/drive/MyDrive/dataset/MostFrequentLocation.csv')

Trên tập dữ liệu của mỗi người dùng - xác định các điểm được ghi nhận nhiều nhất

In [ ]:
#LocationMostAppearOnUser
LocationAppearOnUser = df.groupby('User')['Latitude', 'Longitude'].apply(lambda x: x.value_counts().index[0]).reset_index()
LocationAppearOnUser = LocationAppearOnUser.rename(columns = {0 : 'Lat-Long'})
LocationAppearOnUser

<ipython-input-10-5bd627ed502e>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  LocationAppearOnUser = df.groupby('User')['Latitude', 'Longitude'].apply(lambda x: x.value_counts().index[0]).reset_index()


,User,Lat-Long
0,0.0,"(40.01, 116.315)"
1,1.0,"(40.014, 116.306)"
2,2.0,"(39.926, 116.337)"
3,3.0,"(40.008, 116.32)"
4,4.0,"(40.0, 116.327)"
...,...,...
177,177.0,"(39.938, 116.39)"
178,178.0,"(39.978, 116.332)"
179,179.0,"(40.008, 116.319)"
180,180.0,"(29.017, 115.807)"


In [ ]:
for i in range(len(LocationAppearOnUser)):
  LocationAppearOnUser.loc[i, 'Location_type'] = get_location_type(LocationAppearOnUser.loc[i, 'Lat-Long'][0], LocationAppearOnUser.loc[i, 'Lat-Long'][1])
LocationAppearOnUser

,User,Lat-Long,Location_type
0,0.0,"(40.01, 116.315)",house
1,1.0,"(40.014, 116.306)",residential
2,2.0,"(39.926, 116.337)",residential
3,3.0,"(40.008, 116.32)",university
4,4.0,"(40.0, 116.327)",unclassified
...,...,...,...
177,177.0,"(39.938, 116.39)",bakery
178,178.0,"(39.978, 116.332)",residential
179,179.0,"(40.008, 116.319)",residential
180,180.0,"(29.017, 115.807)",tertiary


In [ ]:
LocationAppearOnUser['Location_type'].unique()

array(['house', 'residential', 'university', 'unclassified', 'parking',
       'kindergarten', 'pedestrian', 'research_institute', 'yes',
       'station', 'tertiary', 'primary', 'massage', 'school',
       'bicycle_parking', 'secondary', 'hospital', 'car', 'gift',
       'common', 'platform', 'hotel', 'attraction', 'research',
       'surveillance', 'trunk', 'cafe', 'post_box', 'garden', 'office',
       'motorway', 'bridge', 'administrative', 'parking_entrance',
       'construction', 'cycleway', 'books', 'playground', 'pitch',
       'restaurant', 'footway', 'living_street', 'theme_park', 'museum',
       'viewpoint', 'government', 'bakery'], dtype=object)

In [ ]:
LocationAppearOnUser['Location_type'].nunique()

47

In [ ]:
def AppearanceFrequency(number):
  user = df[df['User'] == number]
  length = len(user)
  user = user.groupby(['Latitude', 'Longitude']).size().sort_values(ascending=False).reset_index(name='count')
  count = user.loc[0, 'count']
  return (count, count/length)


In [ ]:
for i in range(len(LocationAppearOnUser)):
  (a, b) = AppearanceFrequency(LocationAppearOnUser.loc[i, 'User'])
  LocationAppearOnUser.loc[i, 'Appearence'] = a
  LocationAppearOnUser.loc[i, 'Frequency'] = b
LocationAppearOnUser

,User,Lat-Long,Location_type,Appearence,Frequency
0,0.0,"(40.01, 116.315)",house,8595.0,0.049433
1,1.0,"(40.014, 116.306)",residential,6947.0,0.063965
2,2.0,"(39.926, 116.337)",residential,16156.0,0.065088
3,3.0,"(40.008, 116.32)",university,21795.0,0.044917
4,4.0,"(40.0, 116.327)",unclassified,16926.0,0.038521
...,...,...,...,...,...
177,177.0,"(39.938, 116.39)",bakery,130.0,0.043993
178,178.0,"(39.978, 116.332)",residential,39.0,0.464286
179,179.0,"(40.008, 116.319)",residential,7832.0,0.046235
180,180.0,"(29.017, 115.807)",tertiary,1543.0,0.032714


In [53]:
for i in range(len(LocationAppearOnUser)):
  LocationAppearOnUser.loc[i, 'Lat'] = LocationAppearOnUser.loc[i, 'Lat-Long'][0]
  LocationAppearOnUser.loc[i, 'Long'] = LocationAppearOnUser.loc[i, 'Lat-Long'][1]
LocationAppearOnUser = LocationAppearOnUser.drop(['Lat-Long'], axis=1)
LocationAppearOnUser


,User,Location_type,Appearence,Frequency,Lat,Long
0,0.0,house,8595.0,0.049433,40.010,116.315
1,1.0,residential,6947.0,0.063965,40.014,116.306
2,2.0,residential,16156.0,0.065088,39.926,116.337
3,3.0,university,21795.0,0.044917,40.008,116.320
4,4.0,unclassified,16926.0,0.038521,40.000,116.327
...,...,...,...,...,...,...
177,177.0,bakery,130.0,0.043993,39.938,116.390
178,178.0,residential,39.0,0.464286,39.978,116.332
179,179.0,residential,7832.0,0.046235,40.008,116.319
180,180.0,tertiary,1543.0,0.032714,29.017,115.807


In [55]:
LocationAppearOnUser.to_csv('/content/drive/MyDrive/dataset/LocationAppearOnUser.csv')


Trên toàn bộ tập dữ liệu - xác định thời gian nhiều user nhất cùng xuất hiện trên bản đồ




In [ ]:
UserAndDate=df.drop(['Latitude','Longitude', 'Time', 'Date'],axis=1)
UserAndDate['DateNumber'] = UserAndDate['DateNumber'].apply(np.floor)
UserAndDate

,User,DateNumber
0,0.0,39744.0
1,0.0,39744.0
2,0.0,39744.0
3,0.0,39744.0
4,0.0,39744.0
...,...,...
24708580,181.0,39521.0
24708581,181.0,39521.0
24708582,181.0,39521.0
24708583,181.0,39521.0


In [ ]:
UserDateGroupByDate = UserAndDate.groupby('DateNumber')['User'].nunique().sort_values(ascending=False).reset_index(name='user_count')
UserDateGroupByDate

,DateNumber,user_count
0,39861.0,29
1,39884.0,28
2,39885.0,28
3,39792.0,28
4,39864.0,28
...,...,...
1855,40498.0,1
1856,40497.0,1
1857,40475.0,1
1858,40443.0,1


In [ ]:
from datetime import datetime, timedelta
base_date = datetime(1899, 12, 30)
for i in range(len(UserDateGroupByDate)):
  UserDateGroupByDate.loc[i, 'Date'] = (base_date + timedelta(days=UserDateGroupByDate.loc[i, 'DateNumber'])).date()
UserDateGroupByDate

,DateNumber,user_count,Date
0,39861.0,29,2009-02-17
1,39884.0,28,2009-03-12
2,39885.0,28,2009-03-13
3,39792.0,28,2008-12-10
4,39864.0,28,2009-02-20
...,...,...,...
1855,40498.0,1,2010-11-16
1856,40497.0,1,2010-11-15
1857,40475.0,1,2010-10-24
1858,40443.0,1,2010-09-22


In [ ]:
CrowedDate = UserDateGroupByDate.loc[0, 'DateNumber']
LocationInCrowedDate = df.loc[df['DateNumber'].apply(np.floor)==CrowedDate]

In [ ]:
FirstLocation_OfEachUser_InCrowedDate = LocationInCrowedDate.groupby('User').first().drop(['Date', 'Time', 'DateNumber'],axis=1)
FirstLocation_OfEachUser_InCrowedDate = pd.DataFrame(FirstLocation_OfEachUser_InCrowedDate)

In [ ]:
for i in FirstLocation_OfEachUser_InCrowedDate.index:
  FirstLocation_OfEachUser_InCrowedDate.loc[i, 'Location_type'] = get_location_type(FirstLocation_OfEachUser_InCrowedDate.loc[i, 'Latitude'], FirstLocation_OfEachUser_InCrowedDate.loc[i, 'Longitude'])
FirstLocation_OfEachUser_InCrowedDate

,Latitude,Longitude,Location_type
User,,,
2.0,39.922,116.349,tertiary
3.0,40.011,116.315,house
4.0,39.990,116.332,bicycle_parking
13.0,39.975,116.363,designated
14.0,40.150,116.535,residential
17.0,40.010,116.346,primary
22.0,39.970,116.301,tertiary
24.0,39.991,116.331,bicycle_parking
25.0,39.987,116.302,university


In [ ]:
FirstLocation_OfEachUser_InCrowedDate['Location_type'].unique()

array(['tertiary', 'house', 'bicycle_parking', 'designated',
       'residential', 'primary', 'university', 'it', 'parking',
       'swimming_pool', 'bus_stop', 'hotel', 'secondary'], dtype=object)

In [ ]:
FirstLocation_OfEachUser_InCrowedDate['Location_type'].nunique()

13

In [ ]:
FirstLocation_OfEachUser_InCrowedDate.to_csv('/content/drive/MyDrive/dataset/FirstLocation_OfEachUser_InCrowedDate2.csv')

Trên toàn bộ tập dữ liệu - xác định vị trí nhiều user đi qua nhất trên bản đồ

---



In [ ]:
AlluserLocation = df.drop(['Date', 'Time', 'DateNumber'],axis=1)
AlluserLocation

,User,Latitude,Longitude
0,0.0,39.985,116.318
1,0.0,39.985,116.318
2,0.0,39.985,116.318
3,0.0,39.985,116.318
4,0.0,39.985,116.318
...,...,...,...
24708580,181.0,40.915,111.710
24708581,181.0,40.914,111.710
24708582,181.0,40.912,111.711
24708583,181.0,40.912,111.711


In [ ]:
AlluserLocation=AlluserLocation.drop_duplicates()
AlluserLocation

,User,Latitude,Longitude
0,0.0,39.985,116.318
8,0.0,39.985,116.317
13,0.0,39.985,116.316
18,0.0,39.985,116.315
23,0.0,39.984,116.314
...,...,...,...
24708576,181.0,40.916,111.714
24708577,181.0,40.916,111.713
24708581,181.0,40.914,111.710
24708582,181.0,40.912,111.711


In [ ]:
MostappearlocationCount = AlluserLocation.groupby(['Latitude','Longitude'])['User'].nunique().sort_values(ascending=False).reset_index(name='user_count')
MostappearlocationCount

,Latitude,Longitude,user_count
0,39.975,116.331,144
1,39.975,116.334,137
2,39.975,116.332,137
3,39.975,116.330,136
4,39.975,116.329,133
...,...,...,...
673560,34.774,113.569,1
673561,34.774,113.564,1
673562,34.774,113.552,1
673563,34.774,105.642,1


In [ ]:
MostappearlocationCount = MostappearlocationCount[MostappearlocationCount['user_count'] > 70]
MostappearlocationCount

,Latitude,Longitude,user_count
0,39.975,116.331,144
1,39.975,116.334,137
2,39.975,116.332,137
3,39.975,116.330,136
4,39.975,116.329,133
...,...,...,...
170,39.986,116.361,71
171,39.985,116.343,71
172,39.992,116.328,71
173,39.974,116.328,71


In [ ]:
for i in range(len(MostappearlocationCount)):
  MostappearlocationCount.loc[i, 'Location_type'] = get_location_type(MostappearlocationCount.loc[i, 'Latitude'], MostappearlocationCount.loc[i, 'Longitude'])
MostappearlocationCount

<ipython-input-30-69d6c530cd4a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MostappearlocationCount.loc[i, 'Location_type'] = get_location_type(MostappearlocationCount.loc[i, 'Latitude'], MostappearlocationCount.loc[i, 'Longitude'])


,Latitude,Longitude,user_count,Location_type
0,39.975,116.331,144,yes
1,39.975,116.334,137,primary
2,39.975,116.332,137,tertiary
3,39.975,116.330,136,yes
4,39.975,116.329,133,yes
...,...,...,...,...
170,39.986,116.361,71,trunk
171,39.985,116.343,71,tertiary
172,39.992,116.328,71,mall
173,39.974,116.328,71,yes


In [ ]:
MostappearlocationCount['Location_type'].unique()

array(['yes', 'primary', 'tertiary', 'it', 'government', 'residential',
       'hospital', 'secondary', 'police', 'trunk', 'research_institute',
       'unclassified', 'pitch', 'bicycle_parking', 'electronics',
       'commercial', 'apartments', 'post_box', 'bridge', 'school',
       'cycleway', 'surveillance', 'university', 'parking', 'pedestrian',
       'bank', 'fast_food', 'cafe', 'restaurant', 'attraction',
       'bus_stop', 'mall'], dtype=object)

In [ ]:
MostappearlocationCount['Location_type'].nunique()

32

In [ ]:
MostappearlocationCount.to_csv('/content/drive/MyDrive/dataset/MostappearlocationCount.csv')